# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def color_select(image,threshold_r,threshold_g,threshold_b):
# Use a "bitwise OR" to identify pixels below the threshold
  thresholds = (image[:,:,0] < threshold_r) \
            | (image[:,:,1] < threshold_g) \
            | (image[:,:,2] < threshold_b)
  filtered = image.copy()
  filtered[thresholds] = [0,0,0]
  return filtered




## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
#filter colors exeeding a statistical criteria
def filterImage(image):
  meancolors = np.mean(image,axis=(0,1))
  sdevcolors = np.std(image,axis=(0,1))
#threshold is mean + 1.5 std
  thresholds = meancolors+sdevcolors*1.5 
#to pass yellow blue threshold must be much lower
  filtered_image = color_select(image,thresholds[0],thresholds[1],thresholds[2]*0.4)
  return filtered_image
    
#average similar lines in a line list identified by rho and theta by comparing the theta of the lines.
def average_lines(lines, delta_theta):
  av2_lines=[]
  for i in range(lines.shape[0]):
    j=0
    for rho,theta in lines[i]:
      if theta == 0 and rho == 0:
        continue
    
      av_lines=[]
      av_lines.append((rho, theta))    
      for j in range(lines.shape[0]-1, i, -1):
        for rho2,theta2 in lines[j]:
#         angle is similar
          if theta2 != 0 and rho2 != 0 and abs(theta2-theta) <= delta_theta:
            av_lines.append((rho2, theta2))  
            lines[j] = (0.0)

      av = np.mean(av_lines,axis=0)       
      av2_lines.append((av[0],av[1]))
#      print("averaged",av)
  return av2_lines

##add to a line list identified by rho and theta the ccordinates of the cut of the bottom line or the righ border
def extend_lines_to_bottom(lines, maxY,maxX): 
  lines2 = []
  for point in lines:
    rho = point[0]
    theta = point[1]
    
    #filter strange angles
#    if theta > np.pi * 0.8 and theta < np.pi * 0.2:
#      continue
    
    #x to the left, theta is angle between y axis at the top left corner and the line
    #y at bottom is image.shape[0]
    a = np.cos(theta)
    b = np.sin(theta)
    
    #avoid div by zero
    if a == 0: 
      a = 0.0001
    if b == 0:
      b = 0.0001       
        
    x0 = a*rho
    y0 = b*rho
    
    #calculate length auf line at border of image
    #hit at left border 0 = x0 + lx*(-b) -> lx  = x0/b
    lx = int(x0 / (b))
    #hit at top 0 = y0 + ly*(a) -> lyx  = -y0/a
    ly = int(y0 / (-a))
    #hit at bottom image.shape[0] =y0 + lmaxy*(a) -> lmaxy = (image.shape[0] - y0)/a
    lmaxy = (maxY - y0)/a
    #hit at right border image.shape[1] =x0 + lx*(-b) -> lmaxx = (image.shape[1] - x0)/(-b)
    lmaxx = (maxX - x0)/(-b)
    
    #x at bottom line
    yl = int(y0 + lx*(a))
    xb = int(x0 + lmaxy*(-b))
    yr = int(y0 + lmaxx*(a))

    #hit bottlom line
    if xb >= 0 and xb <= maxX:
      x2 = xb
      y2 = maxY
      #store rho, theta and bottom ccordinates
#    elif xb < 0 :
#      y2 = yl
#      x2 = 0
#    elif xb > maxX :
#      y2 = yr
#      x2 = maxX

#      print("theta:",theta*180/np.pi," x0:",x0," y0:",y0," x2:",x2," y2:",y2)
      lines2.append((rho,theta,x2,y2))
        
  return lines2

#find the cut point of a pair of lines and extend the two lines to the point
def extend_lines_to_cut_point(lines, limitY):
  lines2=[]
  ci = 0
#loop the lines
  for i in range(len(lines)):
    no_cut = True
    rho1,theta1,xb1,yb1 = lines[i]       

    a1 = np.cos(theta1)
    b1 = np.sin(theta1)
    x01 = a1*rho1
    y01 = b1*rho1

    #avoid div by zero
    if a1 == 0: 
      a1 = 0.0001
    if b1 == 0:
      b1 = 0.0001       
    
    j = i +1
#second loop for otherlines lines to find crossings
    for j in range(i+1,len(lines)):    
#      print("cut between {0} and {1}".format(i,j))

      rho2,theta2,xb2,yb2 = lines[j]       
      #rho,theta = av
      a2 = np.cos(theta2)
      b2 = np.sin(theta2)
      x02 = a2*rho2
      y02 = b2*rho2     

#cut of both lines, get the formular
#xc and yc are queal -> y01 + lc1*(a1) = y02 + lc2*(a2) and x01 + lx1*(-b1) = x02 + lx2*(-b2)
#lc1 = (y02 - y01)/a1 + lc2* a2/a1 
#lc1 = (x02 - x01)/(-b1) + lc2* b2/b1 
#lc2 = ((y02 - y01)/a1 + (x02 - x01)/(b1)) / (b2/b1 - a2/a1)

      lc2 = ((y02 - y01)/a1 + (x02 - x01)/b1) / (b2/b1 - a2/a1)
    
      xc = x02 + lc2*(-b2)
      yc = y02 + lc2*(a2)
    
      #cut outside of image?
#      if xc > image.shape[1] or xc < 0:
#        continue
    
#      if yc > image.shape[0] or yc < 0:
#        continue
        
      xc = int(max(min(xc,10000),0.0001))#void int overflow
      yc = int(max(min(yc, 10000),0.0001))
      lines2.append([xb1, yb1, xc, yc])  
      lines2.append([xb2, yb2, xc, yc])
      no_cut = False
        
    if no_cut and len(lines2) == 0:
      #limit the line to a middle area
      xc = x01
      yc = y01
      if y01 <= limitY:
#        print(limitY, yc)
        lc = (limitY -y01) / a1
        xc = int(x01 + lc*(-b1))
        yc = int(y01 + lc*(a1))
        
      lines2.append([xc, yc, xb1, yb1])  
        
#take only the most left and most right lines for defining area of interest
  if len(lines2) > 2:
    mostLeftBottomX = np.min(lines2,axis=0)[0]
    mostRightBottomX = np.max(lines2,axis=0)[0]
#    print("xleft:", mostLeftBottomX, " xright:" ,mostRightBottomX)
    lines3 = []
#copy first most left line    
    for line in lines2:
      if line[0] == mostLeftBottomX:
        lines3.append(line)
        break
#copy first most right line    
    for line in lines2:
      if line[0] == mostRightBottomX:
        lines3.append(line)
        break
    lines2 = lines3                      
                       
  return lines2
    

In [ ]:
import os
print(os.listdir("test_images/"))

#the process chain to draw lane lines on the road
def process_colored_image(image):
#plt.imshow(image)
#plt.show()

#white filter
#  filtered_image = color_select(image,200,200,200)
#yellow color filter
#  filtered_image = color_select(image,200,200,120)

  filtered_image = filterImage(image)
#plt.imshow(filtered_image)
#plt.show()

  gray = grayscale(filtered_image)
#plt.imshow(gray,cmap='gray')
#plt.show()

# Define a kernel size and apply Gaussian smoothing
  kernel_size = 5
  blur_gray = gaussian_blur(gray,kernel_size).copy()
#plt.imshow(blur_gray,cmap='gray' )
#plt.show()

# Define our parameters for Canny and apply
  low_threshold = 200
  high_threshold = 255
  edges = canny(blur_gray, low_threshold, high_threshold).copy()
#  plt.imshow(edges,cmap='gray' )
#  plt.show()

# Define the Hough transform parameters
# Make a blank the same size as our image to draw on
  rho = 1
  theta = np.pi/180
  threshold = 1
  min_line_length = 60
  max_line_gap = 40

# Run Hough on edge detected image
  lines = cv2.HoughLines(edges, rho, theta, min_line_length)

  #max difference in angle for similar lines
  delta_theta = 0.2
  lines_img = np.zeros((*edges.shape, 3), dtype=np.uint8)

  lines = average_lines(lines, delta_theta)
  lines = extend_lines_to_bottom(lines,lines_img.shape[0], lines_img.shape[1])
  lines = extend_lines_to_cut_point(lines,lines_img.shape[0]*0.5)
  for line in lines:
    cv2.line(lines_img, (line[0], line[1]), (line[2], line[3]), [255,0,0], 15)

#  lines_img = hough_lines(edges,rho, theta, threshold, min_line_length, max_line_gap)
#  plt.imshow(lines_img,cmap='gray' )
#  plt.show()

  result = weighted_img(lines_img, image)
  return result

for item in os.listdir("test_images/"):
  #Read in and grayscale the image
  print(item)
  path_to_image = os.path.join("test_images",item)
  image = mpimg.imread(path_to_image)
  result = process_colored_image(image)
  path_to_result = os.path.join("test_images","result_{0}".format(item))
  fig = plt.gcf()
  plt.imshow(result)
  fig.savefig(path_to_result) 
  plt.show()



run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    

    return process_colored_image(image)

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("P1_example.mp4"))

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


There are three major weaknesses of the current implementation:

1. thresholds for color filtering of the image are not fitting to all environments. 
   Even I have taken a statistic, the videos show, that its not too robust to shadows.
   An improvement would be to use thresholds based on a statistic only for the lower image part.
   
2. the thresholds for the HoughLinesalgorithm are very sensitiv  to adjust. 
   If more lines are detected, an additional filter would be required to select the relevant lines that
   reflect the lane lines. I would improve first to create lines only on the lower image part and would filter
   out lines that dont have an range of angle. Something +- 20 degrees around 50 degrees on the left side 
   and 120 degrees on the right side.
   
3. The averaged lane line segments in curves result in lines that do not fit to the real lane lines. 
   Averaging should be reduced to the bottom part of the image to avoid mismatch of the curve segments 
   in the upper image area. again limiting line detection to the lower image part would help. 
   May be averaging of line segments, which not only are similar in the angle, but as well are close to each
   other in the location would help.
  

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))